In [2]:
import chess
import numpy as np
import utils
conf = utils.Config()


def set_piece_on_free_square(board, free_squares, piece):
    '''
    Given the new piece to put and the free squares available, puts the piece in a legal position
    '''
    is_valid = False
    remaining_squares = free_squares.copy()
    c = 0

    # retry until it's valid, stop if there are no more remaining squares (c is a control counter)
    while not is_valid and c<64 and len(remaining_squares)>0:
        c+=1
        
        # randomply choose a square
        square = np.random.choice(remaining_squares)
        remaining_squares.remove(square)
        # put down the piece
        board.set_piece_at(square, piece) 


        if piece.piece_type == chess.KING:
            # opposite check (kings touch)
            if board.status() == chess.STATUS_OPPOSITE_CHECK:    
                is_valid = False
            else:
                is_valid = True
        else:
            # check if it's a valid position
            is_valid = board.is_valid()

        if not is_valid:
            board.remove_piece_at(square) 
    
    if c >= 64 or len(remaining_squares) <= 0:
        # error, makes the whole process restart again
        return None
    else:
        free_squares.remove(square)
        # return the used square
        return square


def set_rand_piece(board, free_squares, all_pieces):
    '''
    Given the free squares and the pieces from which to choose, adds a random piece
    '''

    remaining_pieces = all_pieces.copy()

    # tries with all different pieces until one works
    while len(remaining_pieces) > 0:

        piece = np.random.choice(remaining_pieces)
        remaining_squares = free_squares.copy()
        square = set_piece_on_free_square(board, remaining_squares, piece)

        # set_piece has failed, retry
        if square == None:
            remaining_pieces.remove(piece)
        else:
            # not only valid but also legal position (you have to be able to make moves, not instant end)
            if board.outcome() == None: 
                free_squares.remove(square)
                return True
            # if it's an instant end remove the piece you just placed and try with another piece
            else: 
                board.remove_piece_at(square)
                remaining_pieces.remove(piece)
                
    # if none of the pieces can be placed
    return False


def gen_rand_board_n_pieces(n_pieces, pyece_types=[chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT, chess.PAWN], color=None):
    assert n_pieces > 2, "you cannot have less than three pieces"
    
    flag = False
    while flag == False:
        flag = True
        board = chess.Board()

        # WHITE to move
        board.clear() 
        free_squares = list(range(64))

        set_piece_on_free_square(board, free_squares, chess.Piece(chess.KING, chess.WHITE))
        set_piece_on_free_square(board, free_squares, chess.Piece(chess.KING, chess.BLACK))

        colors = [chess.WHITE, chess.BLACK]
        all_pieces = [chess.Piece(piece, color) for piece in pyece_types for color in colors]

        # randomly choose if first move is black's or white
        if color == None:
            np.random.choice([chess.WHITE, chess.BLACK])
            board.push(chess.Move.null())
            board.clear_stack()
        # else put the chosen color as first player
        else:
            if color == chess.BLACK:
                board.push(chess.Move.null())
                board.clear_stack()

        # put the pieces on the board
        for n in range(n_pieces-2):
            # if at any point set_rand_piece fails, break and restart from scratch the loop
            if flag == False:
                break
            else:
                flag = set_rand_piece(board, free_squares, all_pieces)
        
        if flag == False:
            print("Restarting, cannot reach n_pieces")

    return board

2022-08-18 22:40:17.938197: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 22:40:17.984442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 22:40:17.984795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 22:40:17.985628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

For "ground truth" you can evaluate with tablebase

In [7]:
def generate_dataset(total_games, path, n_pieces_min=3, n_pieces_max=7, piece_types=[chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT, chess.PAWN]):
    '''
    Write in a txt file fen strings of starting endgames positions, given piece types and number of pieces
    '''
    
    with open(path, "w") as f: # to generate the file in case it does not exist
        pass

    with open(path, "a") as f:
        for _ in range(int(total_games/(2*(n_pieces_max+1-n_pieces_min)))):
            # equal distribution of initial move for each color
            for color in [chess.WHITE, chess.BLACK]:
                # equal distribution of piece ranges (from min to max)
                for n_pieces in range(n_pieces_min, n_pieces_max+1):
                    board = gen_rand_board_n_pieces(n_pieces, piece_types, color)
                    f.write(board.fen()+"\n")

In [8]:
# generate_dataset(conf.N_GAMES_ENDGAME_TRAIN, conf.PATH_ENDGAME_TRAIN_DATASET)
generate_dataset(conf.N_GAMES_ENDGAME_EVAL, conf.PATH_ENDGAME_EVAL_DATASET)
# generate_dataset(conf.N_GAMES_ENDGAME_ROOK, conf.PATH_ENDGAME_ROOK, n_pieces_max=3, piece_types=[chess.ROOK])
# generate_dataset(conf.N_GAMES_ENDGAME_TRAIN, conf.PATH_ENDGAME_3_4_pieces, n_pieces_max=4)